In [5]:
import sys
sys.path.append("/Users/aaron/phd/impact-of-contamination-on-taxonomy")

In [6]:
from workflow.fastani_contig_split.d_report_results import FastAniContigSplitReportResultsFastAni
from collections import Counter
from workflow.external.gtdb_metadata import GtdbMetadataR207
from collections import defaultdict


In [19]:
def load_dataframe():
    df_ani = FastAniContigSplitReportResultsFastAni().output().read_cached()
    df_meta = GtdbMetadataR207().output().read_cached()

    return df_meta.merge(df_ani, left_on='gid', right_on='gid', how='inner')

DF_RESULTS = load_dataframe()
DF_RESULTS.head()

,gid,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
0,GCA_000143435.1,2503,99.48,0.52,350,o__Lactobacillales (UID374),191,0.00,1846395,86.436129,...,GCF_001435955.1,97.3962,97.3042,0.8174,0.7677,sp_rep,sp_rep,True,True,True
1,GCA_000153745.1,0,94.58,3.48,615,f__Rhodobacteraceae (UID3356),329,0.00,2113867,92.035190,...,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True
2,GCA_000155005.1,0,98.81,8.44,1171,f__Enterobacteriaceae (UID5124),335,62.64,5486928,72.322300,...,NaN,96.5879,0.0000,0.7680,0.0000,sp_rep,no_af,True,False,False
3,GCA_000161815.1,0,99.08,0.18,625,f__Rhodobacteraceae (UID3375),283,0.00,4116223,87.783094,...,GCF_001681715.1,97.1446,96.9094,0.9309,0.8098,sp_rep,sp_rep,True,True,True
4,GCA_000166735.2,33,95.81,6.07,443,f__Leuconostocaceae (UID486),178,45.83,1936773,84.277582,...,NaN,0.0000,0.0000,0.0000,0.0000,no_ani,no_ani,True,True,True


In [40]:
DF_RESULTS[DF_RESULTS['gid'] == 'GCA_016203935.1']

,gid,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,disc_sp_rep,keep_ani,disc_ani,keep_af,disc_af,keep_type,disc_type,keep_same_as_207,disc_same_as_207,keep_same_as_disc
7282,GCA_016203935.1,0,61.16,0.0,151,k__Bacteria (UID1452),101,0.0,881474,88.133387,...,NaN,0.0,0.0,0.0,0.0,no_af,no_af,True,True,True


In [32]:
import pandas as pd

def get_tax_changes():

    data = defaultdict(lambda: {
        'keep_tax': list(),
        'chim_tax': list(),
    })
    for row in DF_RESULTS.itertuples():

        expected_tax_split = row.expected_tax.split(';')

        cur_row = {
            'gid': row.gid,
            'expected_rank': row.expected_tax,
            'keep_tax': row.keep_type,
            'chim_tax': row.disc_type
        }

        if len(str(row.keep_tax)) > 10:
            keep_tax_split = row.keep_tax.split(';')

            # find the highest rank at which there is a disagreement
            for i, (e, k) in enumerate(zip(expected_tax_split, keep_tax_split)):
                if e != k:
                    cur_row['keep_tax'] = ';'.join(keep_tax_split[i:])
                    break
            else:
                cur_row['keep_tax'] = 'same'


        if len(str(row.disc_tax)) > 10:
            disc_tax_split = row.disc_tax.split(';')

            # find the highest rank at which there is a disagreement
            for i, (e, k) in enumerate(zip(expected_tax_split, disc_tax_split)):
                if e != k:
                    cur_row['chim_tax'] = ';'.join(disc_tax_split[i:])
                    break
            else:
                cur_row['chim_tax'] = 'same'

        data[row.gtdb_taxonomy]['keep_tax'].append(cur_row['keep_tax'])
        data[row.gtdb_taxonomy]['chim_tax'].append(cur_row['chim_tax'])


    rows = list()
    for tax, d_result in data.items():
        sp = tax.split(';')[-1]
        cur_row = {
            'taxonomy': tax,
            'species': sp
        }
        for k in ('keep_tax', 'chim_tax'):
            cur_lst = list()
            cur_cnts = Counter(d_result[k])
            for tax, cnt in sorted(cur_cnts.items(), key=lambda x: x[1], reverse=True):
                cur_lst.append(f'{tax} ({cnt})')
            cur_row[k] = '; '.join(cur_lst)
            if 'same' in cur_cnts and len(cur_cnts) == 1:
                cur_row[f'{k}_all_same'] = True
            else:
                cur_row[f'{k}_all_same'] = False
        rows.append(cur_row)


    return pd.DataFrame(rows)

DF_TAX_CHANGES = get_tax_changes()

DF_TAX_CHANGES.to_csv('/tmp/sp_clustering_tax_changes_grp_by_species.tsv', sep='\t', index=False)

DF_TAX_CHANGES

,taxonomy,species,keep_tax,keep_tax_all_same,chim_tax,chim_tax_all_same
0,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,s__Ligilactobacillus salivarius,same (95),True,same (95),True
1,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s__Amylibacter sp000153745,no_ani (1); same (1),False,no_ani (1); same (1),False
2,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,s__Escherichia coli,same (3721),True,same (3719); no_af (1); no_ani (1),False
3,d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,s__Epibacterium mobile,same (53); no_ani (1),False,same (53); no_ani (1),False
4,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,s__Leuconostoc inhae,no_ani (1); same (1),False,no_ani (1); same (1),False
...,...,...,...,...,...,...
9984,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,s__Faeciplasma avium,same (1),True,same (1),True
9985,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__C...,s__Gallimonas caecicola,no_ani (1),False,no_ani (1),False
9986,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__C...,s__Parachristensenella avicola,no_ani (1),False,no_ani (1),False
9987,d__Bacteria;p__Actinobacteriota;c__Actinomycet...,s__Gulosibacter sp904848455,same (1),True,same (1),True
